# Ml-Pipes

The objective of this project is to present a gentle introduction of how a machine learning model can be trained and deployed. And how [MLFlow](https://www.mlflow.org/), [FastAPI](https://fastapi.tiangolo.com/) and [Docker](https://docs.docker.com/) can facilite a couple aspects of such a task. Within this context, this notebook represents the development part of the model, where a data scientist would create a few models, evaluate it using some metrics and select the best one based in a metric. There are a lot of different machine learning tools that can be used in order to create models. It is important to note that the objective of this notebook is to give a broad overview of and end to end machine learning process, and therefore it is recommended to have the documentation of the frameworks used here as a companion. Also, the README file from the project explains how to reproduce the whole project.

The rest of the notebook is organized as follows
- 1) The Problem;
- 2) Setting up an MLFlow Experiment;
- 3) Training different Models;
- 4) Setting the Best Model to Production.

The code bellow imports everything that will be used throughout the notebook.

In [1]:
# Import Libs
import pandas as pd
pd.set_option('display.max_columns', 500)

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from settings import EXPERIMENT_NAME, TRACKING_URI, FOLDS, CREDIT_CARD_MODEL_NAME,\
     CHAMPION_METRIC, THRESHOLD  # pylint: disable=import-error
from dao.CreditCardDefault \
    import load_creditcard_dataset  # pylint: disable=import-error

from trainers.h2o_automl import H2OClassifier  # pylint: disable=import-error
from trainers.pycaret import PycaretClassifier  # pylint: disable=import-error
from trainers.spark import SparkClassifier


## 1) The problem

The first thing we need to have to build a model is a problem to solve. Here it is used as example the [Credit Card Default from Kagle](https://www.kaggle.com/mlg-ulb/creditcardfraud), where basically the objective if to predict based on a few features whether or not a client will default on its credit card. The taret variable can assume the values 1, for default, and 0 for non default. Therefore it is a binary classification problem.

Bellow the dataset is imported and the first rows of the dataset. Note that the Time column has been removed from the original dataset.

In [2]:
dataset = load_creditcard_dataset()
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
115755,-0.799734,0.736136,2.833526,0.974067,-0.514931,0.792281,-0.370207,0.557209,-0.299558,0.031400,1.320422,0.313413,-0.279858,0.070491,1.552428,0.222578,-0.315646,0.984362,0.252127,0.243629,0.346323,1.113320,-0.278916,0.032092,0.103587,-0.026899,0.391301,0.149521,13.99,0
85889,-6.737663,4.100392,-4.472101,0.398944,-3.263179,-0.853346,-4.868432,-1.311996,-2.131241,-3.351927,-0.124521,1.312198,-2.229997,2.897330,-0.572473,2.478137,2.617134,1.442733,-1.011416,-0.356178,-1.992125,0.446780,0.389360,0.352274,-0.441319,-0.013318,-2.346112,-0.641572,0.76,0
1609,0.995158,-0.621826,-0.325852,-1.356740,-0.686005,-1.435876,0.409797,-0.334338,0.685586,-0.915185,1.855483,1.516669,0.161217,0.678325,0.754820,-0.604068,-0.219514,0.099932,0.984132,0.247418,0.123912,0.111217,-0.238331,0.606093,0.597169,-0.033266,-0.051457,0.024328,160.00,0
74445,-3.085516,-1.232223,0.763441,-0.242930,1.213530,-0.933337,-1.104850,0.669397,0.619235,-0.260568,-1.388942,-0.316119,-0.494229,0.241798,0.993925,1.071331,-0.762457,-0.133627,-1.297000,-0.405427,-0.109123,-0.241966,-0.984082,-0.689215,-0.614096,0.066562,0.645604,-0.315022,11.40,0
5598,-0.094255,0.373340,1.175382,0.627407,-0.160934,0.310964,0.927078,-0.983293,0.928696,1.421688,0.319144,-2.850179,1.707957,0.491084,-0.855800,-2.854832,1.056708,0.772647,-0.624793,-0.585970,-0.559566,-0.157548,0.158081,-0.007013,-1.211904,-0.627710,-0.852221,-0.525718,76.19,0


## 2) Setting up an MLFlow Experiment

Now that a problem has been stated and some data to help solving the problem has been gathered, the next step is to setup a MLFlow experiment to log our models. **MLFlow is built upon the concept of experiments. A experiment is a series of fits, where parameters, metrics, models and artifacts can be associated with the respective fit (in an machine learning package agnostic way).**

The code bellow tries to create an experiment, if that experiments already existis then it sets the experiment to the active one.
 

In [3]:
TRACKING_URI

'http://127.0.0.1:5000'

In [4]:
EXPERIMENT_NAME='teste4'
# TRACKING_URI='http://127.0.0.1:5000'

In [5]:
# mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_tracking_uri(TRACKING_URI)
try:
    experiment = mlflow.create_experiment(EXPERIMENT_NAME)
except Exception:
    client = MlflowClient()
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

mlflow.set_experiment(EXPERIMENT_NAME)

## 3) Training different Models

The ext step if to train, evaluate and log a few different models. In order to demonstrate that MLFlow allows us to use different machine learning packages we will train an H2O autoML and SkLearn models (using pycaret). Now is the time where MLFlow is put into action: For each model that if fitted it will be logged a few parameters, metrics, artifacts and the models it self. To understand how this is done it checkout the classifiers definitions in `src/trainers/` folder and the [MLFlow Logging Documentaion](https://www.mlflow.org/docs/latest/tracking.html#logging-data-to-runs), ot all happens inside the `mlflow.start_run()` context manager. 

The next cells will train different classifiers. Once they finish running you can deploy the [MLFlow Tracking UI](https://www.mlflow.org/docs/latest/tracking.html#tracking-ui) by executing `mlflow ui -p 5000 --backend-store-uri sqlite:///mlruns.db` in the terminal inside the `src/` folder. and see the results at [127.0.0.1:5000](127.0.0.1:5000).

In [6]:
H2OClassifier(
    run_name='H2O',
    max_mem_size='3G',
    threshold=THRESHOLD,
    df=dataset,
    target_col='Class',
    sort_metric='aucpr',
    max_models=8,
    max_runtime_secs=10,
    nfolds=FOLDS,
    seed=90
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /media/vinicius/Dados/poetry/virtualenvs/ml-pipes-VBbH4xSK-py3.8/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpu6_kf566
  JVM stdout: /tmp/tmpu6_kf566/h2o_vinicius_started_from_python.out
  JVM stderr: /tmp/tmpu6_kf566/h2o_vinicius_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_vinicius_ebckxm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
13:28:45.222: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%
Could not find exact threshold 0.5; using closest threshold found 0.45161522192663917.
Could not find exact threshold 0.5; using closest threshold found 0.45161522192663917.
Could not find exact threshold 0.5; using closest threshold found 0.45161522192663917.
Could not find exact threshold 0.5; using closest threshold found 0.45161522192663917.
Could not find exact threshold 0.5; using closest threshold found 0.45161522192663917.


Exception: [Errno 13] Permission denied: '/usr/app'

In [7]:
PycaretClassifier(
        experiment_name=EXPERIMENT_NAME,
        run_name='Pycaret2',
        sort_metric='precision',
        df=dataset,
        target='Class',
        threshold=THRESHOLD,
        n_best_models=3,
        data_split_stratify=True,
        nfolds=FOLDS,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        session_id=54321
)

,Parameters
alpha,1.0
class_weight,None
copy_X,True
fit_intercept,True
max_iter,None
normalize,False
random_state,54321
solver,auto
tol,0.001


Error in logging parameter for                                 pycaret_precision_2
[Errno 2] No such file or directory: 'Hyperparameters.png'


In [6]:
SparkClassifier(
    df = dataset,
    target_col = 'Class',
    run_name = 'spark_classifier',
    max_mem_size = 4,
    n_cores = 4,
    seed = 90
)

## 4) Setting the Best Model to Production

The final step in this notebook if to set to production the model with the best selected metric, imported as `CHAMPION_METRIC`. This is done to show is is possible to create an automated workflow using MLFlow to deplot a model. However it is also possible to deplot the model using the [UI server](https://www.mlflow.org/docs/latest/model-registry.html#ui-workflow).

Once this is done you can return to the README file to check how the model is now deployed.

In [10]:
# Getting The best Model according to CHAMPION_METRIC
champion = MlflowClient().search_runs(
    experiment_ids=[
        str(
            mlflow.get_experiment_by_name(name=EXPERIMENT_NAME).experiment_id
        )
    ],
    run_view_type=ViewType.ALL,
    order_by=[f"metrics.{CHAMPION_METRIC} DESC"],
    max_results=1
)
run_id = champion[0].info.run_id

# Registering Model in model registery
model = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=CREDIT_CARD_MODEL_NAME
)

# Setting version 1
MlflowClient().update_model_version(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    description='Deploying model with model registery'
)

# Setting it to production
MlflowClient().transition_model_version_stage(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    stage="Staging"
)

Registered model 'CreditCardDefault' already exists. Creating a new version of this model...
2021/05/19 09:02:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CreditCardDefault, version 3
Created version '3' of model 'CreditCardDefault'.


<ModelVersion: creation_timestamp=1621425758707, current_stage='Staging', description='Deploying model with model registery', last_updated_timestamp=1621425758825, name='CreditCardDefault', run_id='c325b227a2d24a3994ca8e75b0201117', run_link='', source='/media/vinicius/Dados/projects/ml-pipes/mlflow_artifact_store/1/c325b227a2d24a3994ca8e75b0201117/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>